In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@e48186f

In [2]:
import spark.implicits._
// Data
case class Prices(product_id: Int, start_date: java.sql.Date, end_date: java.sql.Date, price: Int)
case class UnitsSold(product_id: Int, purchase_date: java.sql.Date, units: Int)
val data = Seq[(Int, String, String, Int)](
  (1, "2019-02-17", "2019-02-28", 5), 
  (1, "2019-03-01", "2019-03-22", 20), 
  (2, "2019-02-01", "2019-02-20", 15), 
  (2, "2019-02-21", "2019-03-31", 30)
)
val data2 = Seq[(Int, String, Int)](
  (1, "2019-02-25", 100), 
  (1, "2019-03-01", 15), 
  (2, "2019-02-10", 200), 
  (2, "2019-03-22", 30)
)


val pricesDS: Dataset[Prices] = data.toDF("product_id", "start_date", "end_date", "price")
  .withColumn("start_date", to_date(col("start_date")))
  .withColumn("end_date", to_date(col("end_date")))
  .as[Prices]
val unitsSoldDS: Dataset[UnitsSold] = data2.toDF("product_id", "purchase_date", "units")
  .withColumn("purchase_date", to_date(col("purchase_date")))
  .as[UnitsSold]

pricesDS.show()
unitsSoldDS.show()

+----------+----------+----------+-----+
|product_id|start_date|  end_date|price|
+----------+----------+----------+-----+
|         1|2019-02-17|2019-02-28|    5|
|         1|2019-03-01|2019-03-22|   20|
|         2|2019-02-01|2019-02-20|   15|
|         2|2019-02-21|2019-03-31|   30|
+----------+----------+----------+-----+

+----------+-------------+-----+
|product_id|purchase_date|units|
+----------+-------------+-----+
|         1|   2019-02-25|  100|
|         1|   2019-03-01|   15|
|         2|   2019-02-10|  200|
|         2|   2019-03-22|   30|
+----------+-------------+-----+



import spark.implicits._
// Data

defined class Prices
defined class UnitsSold
data: Seq[(Int, String, String, Int)] = List(
  (1, "2019-02-17", "2019-02-28", 5),
  (1, "2019-03-01", "2019-03-22", 20),
  (2, "2019-02-01", "2019-02-20", 15),
  (2, "2019-02-21", "2019-03-31", 30)
)
data2: Seq[(Int, String, Int)] = List(
  (1, "2019-02-25", 100),
  (1, "2019-03-01", 15),
  (2, "2019-02-10", 200),
  (2, "2019-03-22", 30)
)
pricesDS: Dataset[Prices] = [product_id: int, start_date: date ... 2 more fields]
unitsSoldDS: Dataset[UnitsSold] = [product_id: int, purchase_date: date ... 1 more field]

Write a solution to find the average selling price for each product. average_price should be rounded to 2 decimal places. If a product does not have any sold units, its average selling price is assumed to be 0.

Return the result table in any order.

The result format is in the following example.

| product_id | average_price |
|------------|---------------|
| 1          | 6.96          |
| 2          | 16.96         |

In [3]:
pricesDS.join(
        unitsSoldDS, 
        (unitsSoldDS("product_id") === pricesDS("product_id")) &&
        (unitsSoldDS("purchase_date") >= pricesDS("start_date")) &&
        (unitsSoldDS("purchase_date") <= pricesDS("end_date")), 
        "inner"
    )
    .select(pricesDS("product_id").as("product_id"), col("price"), col("units"))
    .groupBy(col("product_id"))
    .agg(round(sum(col("price") * col("units"))/sum(col("units")), 2).as("average_price"))
    .show()


+----------+-------------+
|product_id|average_price|
+----------+-------------+
|         1|         6.96|
|         2|        16.96|
+----------+-------------+



In [4]:
spark.stop